In [1]:
from datasets import load_dataset, Dataset, load_metric
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoConfig, LlamaTokenizer
#from transformers import AdapterConfig, AdapterType
from transformers import TrainingArguments, Trainer, BitsAndBytesConfig, DataCollatorWithPadding
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import csv
import re
import nltk
import pickle
import os
import time
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.translate.bleu_score import corpus_bleu
from sacrebleu.metrics import BLEU, CHRF
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, hamming_loss, accuracy_score
from peft import PeftModel, PeftConfig
import sacrebleu
import logging
import bitsandbytes as bnb
from bitsandbytes.optim import GlobalOptimManager
#from trl import SFTTrainer
#import optimum
#import adapter-transformers
from torch.nn import DataParallel
logging.basicConfig(level=logging.INFO)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def format_response(response: str) -> str:
    # Add a newline before and after each list item
    formatted_response = re.sub(r'(\d+\.)', r'\n\1\n', response)

    # Add a newline before the start of the numbered list
    formatted_response = re.sub(r'(\:\s*)\n(\d+\.)', r'\1\n\n\2', formatted_response)

    # Remove unwanted newlines following list numbers
    formatted_response = re.sub(r'(\d+\.)\n\s+', r'\1 ', formatted_response)

    # Identify the last item of the list and add an extra paragraph break if needed
    formatted_response = re.sub(r'(\d+\.\s.*?\.)\s+(\w)', r'\1\n\n\2', formatted_response)

    # Clean up: Ensure there are not more than two consecutive newlines
    formatted_response = re.sub(r'\n{3,}', '\n\n', formatted_response)
    formatted_response = formatted_response.strip()

    return formatted_response

def adjust_list_spacing(formatted_response: str) -> str:
    # Correctly format each list item with a single line break
    # Replace multiple newlines with a single newline after list numbers
    adjusted_response = re.sub(r'(\d+\.)\s*\n+', r'\1 ', formatted_response)

    # Add a newline after each list item
    adjusted_response = re.sub(r'(\d+\.[^\n]*)(?=\n\d+|$)', r'\1\n', adjusted_response)

    # Special handling for the last list item: Check if there is more content after it
    last_item_match = re.search(r'(\d+\.\s.*?)(?=\n\d+|$)', adjusted_response, re.DOTALL)
    if last_item_match:
        last_item = last_item_match.group(1)
        # If the last item is followed by more content, add an extra paragraph break
        if re.search(r'\.\s', last_item.split('.')[1]):
            adjusted_response = re.sub(last_item + r'\n', last_item + '\n\n', adjusted_response, 1, re.DOTALL)

    return adjusted_response.strip() 


In [3]:
#BNB_CONFIG = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4")
#model_save_path1 = "/workspace/axolotl/merged_model"
model_chat = "audichandra/Ind_chatbot"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = AutoModelForCausalLM.from_pretrained(model_chat
                                              , torch_dtype=torch.bfloat16
                                              #, device_map="auto", pad_token_id=0 
                                              #, attn_implementation="flash_attention_2"  
                                              , cache_dir="/workspace"
                                              #, quantization_config=BNB_CONFIG
                                             ).to(device)
model_id = "Ichsan2895/Merak-7B-v4"
tokenizer = LlamaTokenizer.from_pretrained(model_id)

def generate_response(question: str) -> str:
    chat = [
      {"role": "system", "content": "Ada yang bisa saya bantu?"},
      {"role": "user", "content": question},
    ]

    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)

    inputs = inputs.to(device)  # Ensure inputs are on the same device as the model

    with torch.no_grad():
        outputs = model1.generate(**inputs, max_new_tokens=512)
        response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        assistant_start = f'''{question} \n assistant\n '''
        response_start = response.find(assistant_start)
        response_formatted = format_response(response[response_start + len(assistant_start):].strip())
        final_response = adjust_list_spacing(response_formatted)
        return final_response


# Use the functions together
start_time = time.time()
prompt = "bagaimana saya dapat membatalkan pembelian saya?"
print(generate_response(prompt))

end_time = time.time()
elapsed_time = end_time - start_time 
print(f"Elapsed time: {elapsed_time} seconds")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Saya telah mengambil bahwa Anda mencari bantuan dengan membatalkan pembelian Anda. Untuk membatalkan pembelian, silakan ikuti langkah-langkah ini: 

1. Masuk ke akun Anda di situs kami.

2. Pergi ke bagian "Order History" atau "My Orders." 

3. Temukan pembelian yang ingin Anda batalkan dan klik di atasnya.

4. Cari pilihan "Batal Pembelian" dan pilih itu. 

5. Ikuti instruksi tambahan yang disediakan oleh sistem untuk menyelesaikan proses pembatalan.

Jika Anda menghadapi kesulitan atau memiliki pertanyaan lebih lanjut, jangan ragu untuk menghubungi tim pendukung pelanggan kami. Mereka tersedia {{Customer Support Hours}} di {{Customer Support Phone Number}} atau melalui Chat Langsung di website kami di {{Website URL}}. Kami di sini untuk membantu Anda dan memastikan pengalaman pembatalan yang halus.
Elapsed time: 102.7834701538086 seconds


In [5]:
start_time = time.time() 
prompt = "bagaimana saya bisa melacak pesanan saya?"
print(generate_response(prompt))

end_time = time.time()
elapsed_time = end_time - start_time 
print(f"Elapsed time: {elapsed_time} seconds")

Terima kasih sudah menghubungi! Saya di sini untuk membantu Anda dalam melacak pesanan Anda. Untuk melacak pesanan Anda, Anda dapat mengunjungi situs web kami dan menavigasi ke bagian "Order Tracking." Di sana, Anda akan diminta untuk memasukkan nomor pesanan Anda atau rincian relevan lainnya. Setelah Anda memasukkan informasi yang diperlukan, sistem akan menampilkan status pesanan Anda saat ini dan memperbarui Anda pada kemajuan. Jika Anda memiliki pertanyaan lebih lanjut atau membutuhkan bantuan dengan proses pelacakan, jangan ragu untuk bertanya. Saya di sini untuk membantu Anda setiap langkah dari jalan!
Elapsed time: 25.907153844833374 seconds


In [6]:
start_time = time.time() 
prompt = "bagaimana saya bisa mengembalikan barang dan apakah ketentuan pengembaliannya?"
print(generate_response(prompt))

end_time = time.time()
elapsed_time = end_time - start_time 
print(f"Elapsed time: {elapsed_time} seconds")

Saya telah mengambil bahwa Anda mencari bantuan dengan memulihkan barang dan ingin tahu tentang ketentuan pengembalian dana. Untuk memulai proses pengembalian dana, silakan ikuti langkah-langkah ini: 

1. Hubungi tim pendukung pelanggan kami melalui telepon atau obrolan langsung di situs web kami.

2. Berikan perincian yang diperlukan, seperti nomor pesanan Anda atau informasi relevan lainnya. 

3. Jelaskan alasan untuk meminta pengembalian dana dan menyediakan dokumentasi atau bukti yang diperlukan.

4. Tim pendukung pelanggan kami akan meninjau permintaan Anda dan membimbing Anda melalui proses pengembalian dana. 

5. Setelah pengembalian dana disetujui, Anda akan menerima instruksi tentang cara melanjutkan dengan pengembalian dana.

6. Jika Anda memiliki pertanyaan atau kekhawatiran lebih lanjut, jangan ragu untuk menjangkau tim pendukung pelanggan kami untuk bantuan. Ketentuan pengembalian dana mungkin berbeda - beda bergantung pada produk atau jasa yang Anda beli. Untuk mendapatka

In [7]:
start_time = time.time() 
prompt = "saya ada masalah dengan pembayaran, apakah bisa kamu bantu?"
print(generate_response(prompt))

end_time = time.time()
elapsed_time = end_time - start_time 
print(f"Elapsed time: {elapsed_time} seconds")

Tentu saja! Saya di sini untuk membantu Anda dengan masalah yang Anda hadapi dengan pembayaran Anda. Untuk membantu Anda lebih baik, dapatkah Anda memberikan lebih banyak perincian tentang problem spesifik yang sedang Anda hadapi? Ini akan memungkinkan saya untuk memberikan Anda dengan solusi yang paling akurat dan efektif.
Elapsed time: 14.181119441986084 seconds


In [8]:
start_time = time.time() 
prompt = "bagaimana saya bisa mengubah password saya?"
print(generate_response(prompt))

end_time = time.time()
elapsed_time = end_time - start_time 
print(f"Elapsed time: {elapsed_time} seconds")

Tentu saja! Saya di sini untuk membantu Anda dengan mengubah password Anda. Untuk membuat proses sehalus mungkin, silakan ikuti langkah-langkah ini: 

1. Masuk ke akun Anda pada platform kami.

2. Navigasi ke pengaturan akun atau bagian profil Anda. 

3. Cari pilihan untuk mengubah atau mereset sandi Anda.

4. Ikuti instruksi pada layar untuk membuat sandi baru yang aman. 

5. Pastikan untuk memilih sandi yang kuat dan unik yang mencakup kombinasi huruf, angka, dan karakter khusus.

6. Setelah Anda membuat sandi baru, simpan perubahan dan periksa apakah Anda dapat masuk ke akun Anda menggunakan sandi baru. Jika Anda menghadapi kesulitan apa pun atau memiliki pertanyaan lebih lanjut, jangan ragu untuk membiarkan saya tahu. Saya di sini untuk membantu Anda setiap langkah dari jalan!
Elapsed time: 35.469945192337036 seconds


In [9]:
start_time = time.time() 
prompt = "apakah ada diskon atau promo yang berlaku?"
print(generate_response(prompt))

end_time = time.time()
elapsed_time = end_time - start_time 
print(f"Elapsed time: {elapsed_time} seconds")

Tentu saja! Saya di sini untuk membantu Anda dalam menemukan diskon atau promosi yang berlaku. Tolong berikan saya rincian lebih lanjut tentang produk atau jasa yang Anda minati, dan saya akan melakukan yang terbaik untuk membantu Anda menemukan penawaran yang sesuai.
Elapsed time: 10.407901048660278 seconds


In [10]:
start_time = time.time() 
prompt = "saya mengalami masalah dengan website anda, apakah anda bisa bantu?"
print(generate_response(prompt))

end_time = time.time()
elapsed_time = end_time - start_time 
print(f"Elapsed time: {elapsed_time} seconds")

Tentu saja! Saya di sini untuk membantu Anda dengan masalah yang Anda hadapi dengan website kami. Untuk membantu Anda lebih baik, dapatkah Anda memberikan lebih banyak perincian tentang problem spesifik yang sedang Anda hadapi? Informasi ini akan memungkinkan saya untuk menyediakan Anda dengan bimbingan yang paling akurat dan efektif. Terima kasih sudah menghubungi kami, dan kami menghargai kesabaranmu.
Elapsed time: 17.640464067459106 seconds


In [11]:
start_time = time.time() 
prompt = "bagaimana cara membuat akun baru?"
print(generate_response(prompt))

end_time = time.time()
elapsed_time = end_time - start_time 
print(f"Elapsed time: {elapsed_time} seconds")

Saya senang untuk membantu! Membuat akun baru dengan kami adalah proses yang mudah. Berikut panduan langkah demi langkah untuk membantu Anda: 

1. Kunjungi situs web kami di {{Website URL}}.

2. Cari tombol "Sign Up" atau "Create Account" dan klik di atasnya. 

3. Isi informasi yang diperlukan, seperti nama lengkap, alamat email, dan sandi yang kuat.

4. Periksa kembali rincian yang Anda masukkan untuk memastikan keakuratannya. 

5. Setujui persyaratan dan kondisi kami dan privasi policy.

6. Klik tombol "Create Account" atau "Sign Up" untuk menyelesaikan proses penciptaan akun. Setelah Anda menyelesaikan langkah-langkah ini, Anda akan menerima email konfirmasi dengan instruksi tentang cara memverifikasi akun Anda. Jika Anda menghadapi kesulitan apa pun atau memiliki pertanyaan lebih lanjut, jangan ragu untuk menjangkau tim pendukung pelanggan kami. Mereka tersedia {{Customer Support Hours}} di {{Customer Support Phone Number}} atau melalui Chat Langsung di website kami di {{Website UR

In [12]:
start_time = time.time() 
prompt = "saya ada keluhan, bagaimana cara mengajukannya"
print(generate_response(prompt))

end_time = time.time()
elapsed_time = end_time - start_time 
print(f"Elapsed time: {elapsed_time} seconds")

Saya menyesal mendengar bahwa Anda memiliki keluhan. Umpan balik Anda sangat berharga bagi kami, dan kami ingin memastikan bahwa kami mengatasi kekhawatiran Anda dengan tepat. Untuk mengajukan keluhan, Anda dapat menghubungi tim pendukung pelanggan kami melalui telepon, email, atau obrolan langsung di situs web kami. Mereka akan membimbing Anda melalui proses dan membantu Anda mengajukan keluhan secara efektif. Yakinlah, kami menanggapi keluhan dengan serius dan akan menyelidiki masalah ini dengan saksama. Terima kasih untuk membawa ini ke perhatian kami.
Elapsed time: 24.867743730545044 seconds


In [13]:
start_time = time.time() 
prompt = "saya ingin menghubungi customer service?"
print(generate_response(prompt))

end_time = time.time()
elapsed_time = end_time - start_time 
print(f"Elapsed time: {elapsed_time} seconds")

Terima kasih sudah menghubungi! Saya di sini untuk membantu Anda dalam menghubungi tim layanan pelanggan kami. Untuk menghubungi mereka, Anda dapat menghubungi mereka melalui telepon di {{Customer Support Phone Number}} atau melalui fitur Chat Live di website kami di {{Website URL}}. Tim kami tersedia untuk membantu Anda dengan pertanyaan atau kekhawatiran yang mungkin Anda miliki. Bagaimana saya bisa membantu Anda lebih lanjut?
Elapsed time: 17.295284748077393 seconds


In [14]:
start_time = time.time() 
prompt = "bagaimana cara membeli barang anda?"
print(generate_response(prompt))

end_time = time.time()
elapsed_time = end_time - start_time 
print(f"Elapsed time: {elapsed_time} seconds")

Terima kasih atas minat Anda dalam membeli barang kami! Saya di sini untuk membimbing Anda melalui proses dan membuatnya sehalus mungkin. Untuk memulai, bisakah Anda memberi saya rincian lebih lanjut tentang barang yang ingin Anda beli? Ini akan memungkinkan saya untuk memberikan Anda dengan informasi yang paling akurat dan membantu.
Elapsed time: 13.417717933654785 seconds


In [15]:
start_time = time.time() 
prompt = "bagaimana cara untuk review produk ini?"
print(generate_response(prompt))

end_time = time.time()
elapsed_time = end_time - start_time 
print(f"Elapsed time: {elapsed_time} seconds")

Terima kasih atas minat Anda dalam meninjau produk kami! Untuk meninggalkan ulasan, Anda dapat mengunjungi situs web kami dan menavigasi halaman produk. Di sana, Anda akan menemukan bagian khusus di mana Anda dapat berbagi pikiran dan pengalaman Anda dengan produk. Ulasan Anda akan membantu pelanggan lain membuat keputusan yang terinformasi dan membantu kami meningkatkan penawaran kami. Jika Anda memiliki pertanyaan lebih lanjut atau membutuhkan bantuan dengan proses review, jangan ragu untuk bertanya!
Elapsed time: 21.25312566757202 seconds
